In [1]:
# Labeling the data
import numpy as np
import matplotlib.pyplot as plt
import os
import tensorflow as tf
from PIL import Image

from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input

from tensorflow.keras.utils import normalize, to_categorical

2023-04-03 06:18:47.201556: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-03 06:18:47.252293: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-03 06:18:47.253472: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-03 06:18:48.122385: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
image_directory='Data'
Fake=os.listdir(image_directory+ '/Fake')
Real=os.listdir(image_directory+ '/Real')

print('Fake Images: ', len(Fake))
print('Real Images: ',len(Real))

Fake Images:  100
Real Images:  1000


In [3]:
dataset=[]
label=[]

INPUT_SIZE=224
# Set image dimensions and batch size
img_height = 224
img_width = 224
batch_size = 32

In [4]:
for i, image_name in enumerate(Fake):
    if (image_name.split('.')[1] == 'jpg'):
        image_path = os.path.join(image_directory, 'Fake', image_name)
        image = Image.open(image_path)
        image = image.resize((INPUT_SIZE, INPUT_SIZE))
        dataset.append(np.array(image))
        label.append(0)


In [5]:
for i, image_name in enumerate(Real):
    if (image_name.split('.')[1] == 'png'):
        image_path = os.path.join(image_directory, 'Real', image_name)
        image = Image.open(image_path)
        image = image.resize((INPUT_SIZE, INPUT_SIZE))
        dataset.append(np.array(image))
        label.append(1)

In [6]:
dataset=np.array(dataset)
label=np.array(label)

print('Dataset: ',len(dataset))
print('Label: ',len(label))

Dataset:  1100
Label:  1100


In [7]:
#stratified split
X_train, X_test, y_train, y_val = train_test_split(dataset, label, test_size=0.2, random_state=42, stratify=label)

In [8]:
X_train = normalize(X_train, axis=1)
X_val = normalize(X_test, axis=1)

In [9]:
X_train.shape

(880, 224, 224, 3)

In [10]:
X_val.shape

(220, 224, 224, 3)

In [11]:
# initializing model with weights='imagenet'i.e. we are carring its original weights
model_resnet=ResNet50(weights='imagenet')

In [12]:
# display the summary to see the properties of the model
model_resnet.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [13]:
# Modelling WITH Transfer Learning

# Here we will prepare model as per our requirements

print("Summary of Custom ResNet50 model.\n")
print("1) We setup input layer and 2) We removed top (last) layer. \n")

Summary of Custom ResNet50 model.

1) We setup input layer and 2) We removed top (last) layer. 



In [19]:
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model


In [20]:
resnet_model = ResNet50(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# add a global spatial average pooling layer
x = resnet_model.output
x = GlobalAveragePooling2D()(x)

# add a fully-connected layer
x = Dense(1024, activation='relu')(x)

# add the final output layer with softmax activation
predictions = Dense(2, activation='softmax')(x)

# create the full model
model = Model(inputs=resnet_model.input, outputs=predictions)

In [22]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [23]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))


Epoch 1/10


ValueError: in user code:

    File "/home/codespace/.python/current/lib/python3.10/site-packages/keras/engine/training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "/home/codespace/.python/current/lib/python3.10/site-packages/keras/engine/training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/codespace/.python/current/lib/python3.10/site-packages/keras/engine/training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "/home/codespace/.python/current/lib/python3.10/site-packages/keras/engine/training.py", line 1051, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/home/codespace/.python/current/lib/python3.10/site-packages/keras/engine/training.py", line 1109, in compute_loss
        return self.compiled_loss(
    File "/home/codespace/.python/current/lib/python3.10/site-packages/keras/engine/c

In [14]:
# initialize the transfer model ResNet50 with appropriate properties per our need.
# we are passing paramers as following
# 1) weights='imagenet' - Using this we are carring weights as of original weights.
# 2) input_tensor to pass the ResNet50 using input_tensor
# 3) we want to change the last layer so we are not including top layer
# resnet_model=ResNet50(weights='imagenet',include_top=False)
resnet_model = ResNet50(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# See the summary of the model with our properties.
resnet_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [15]:
resnet_model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [16]:
# Fit the Model

resnet_model.fit(X_train,y_train,epochs=20,batch_size=64,verbose=True,validation_data=(X_val,y_val))

Epoch 1/20


ValueError: in user code:

    File "/home/codespace/.python/current/lib/python3.10/site-packages/keras/engine/training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "/home/codespace/.python/current/lib/python3.10/site-packages/keras/engine/training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/codespace/.python/current/lib/python3.10/site-packages/keras/engine/training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "/home/codespace/.python/current/lib/python3.10/site-packages/keras/engine/training.py", line 1051, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/home/codespace/.python/current/lib/python3.10/site-packages/keras/engine/training.py", line 1109, in compute_loss
        return self.compiled_loss(
    File "/home/codespace/.python/current/lib/python3.10/site-packages/keras/engine/c